# Key papers

**TOPICS OF INTEREST** - from the meeting in January

* inflammation aging chronic (2004) - 13k papers
* genome editing / manipulation, CRISPR - 13k papers
* induced stem cells - 73k papers - 3h for calculating co-citations
* single-cell sequencing (2012) - 3k papers
* ATAC-seq (2015) - 276 papers
* immunomodulation cancer - 71k papers
* Telomere Theories of Aging - ??
* mTOR pathway - 14255
* autophagy - ??
* Calorie restriction - 3933

Complement Factor H + Age-Related Mascular Degeneration - investigate

A file with citation stats is required to run several cells of this notebook successfully.
The file can be obtained with the following commands if user has access to `pubmed` database (use crawler).

1. Run `psql`.
2. Run following commands in the `psql` prompt:
    * `\f ','`
    * `\a`
    * `\t`
    * `\o '/path/to/the/file'`
    * `SELECT C.pmid_cited AS pmid, P.year, COUNT(1) AS count`
    * `FROM Citations C`
    * `JOIN Publications P`
    * `ON C.pmid_citing = P.pmid`
    * `GROUP BY C.pmid_cited, P.year;`
    * `\o`
3. Store `/path/to/the/file` in the `CITATION_STATS_FILE` variable in the following cell.

**Issues**:

1. Some information in tooltips with long titles may occur out of plot bounds.
2. How should I place articles with the same year? (currently y-axis position is random in [0,1]...)
3. Some research on clustering algorithms is needed! (also `networkx.algorithms.community`)

**Functions**:

1. Subtopic Analysis based on co-citation graph clustering
2. Top Cited Papers detection (overall and for certain year)
3. Citation Dynamics for a certain article

## Search Terms

In [1]:
SEARCH_TERMS = ['human', 'aging']

In [2]:
from importlib import reload
import logging
reload(logging)

import re
import gc
import ipywidgets as widgets
import networkx as nx
import numpy as np
import pandas as pd
import psycopg2 as pg_driver

from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, LabelSet, OpenURL, TapTool, CustomJS
from bokeh.plotting import figure, show, output_notebook
from IPython.display import display
from matplotlib import pyplot as plt
%matplotlib inline

from Bio import Entrez

Entrez.email = 'nikolay.kapralov@gmail.com'
PUBMED_ARTICLE_BASE_URL = 'https://www.ncbi.nlm.nih.gov/pubmed/?term='

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.DEBUG)
output_notebook()

Loading BokehJS ...

2019-05-02 18:46:56,052 DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
class KeyPaperAnalyzer:       
    def __init__(self):
        self.conn = pg_driver.connect(dbname='pubmed', user='biolabs', password='pubtrends', host='localhost')
        self.cursor = self.conn.cursor()
        
    def load_publications(self):
        logging.info('Loading publication data')
        
        query = '''
        SELECT pmid, title, year
        FROM Publications
        WHERE pmid = ANY(%s);
        '''
        
        with self.conn:
            self.cursor.execute(query, (self.pmids,))
        pub_data = []
        for row in self.cursor:
            pub_data.append(list(row))
        self.pub_df = pd.DataFrame(pub_data, columns=['pmid', 'title', 'year'])
        logging.info(f'Found {len(self.pub_df)} publications in the local database')
            
    def load_cocitations(self):
        logging.info('Calculating co-citations for selected articles')

        # Optimize WHERE with JOIN (VALUES ... ) AS 
        # See https://pgday.ru/files/pgmaster14/max.boguk.query.optimization.pdf
        # Don't use LIMIT here because it is performed after everything is computed in Postgresql
        values = ', '.join(['({})'.format(i) for i in sorted(self.pmids)])
        query = re.sub('\$VALUES\$', values, '''
        SELECT C1.pmid_citing, C1.pmid_cited, C2.pmid_cited, P.year
        FROM Citations C1
        JOIN (VALUES $VALUES$) AS C1T(pmid_cited) ON (C1.pmid_cited = C1T.pmid_cited)
        JOIN Citations C2
        JOIN (VALUES $VALUES$) AS C2T(pmid_cited) ON (C2.pmid_cited = C2T.pmid_cited)
        ON C1.pmid_citing = C2.pmid_citing AND C1.pmid_cited < C2.pmid_cited
        JOIN Publications P
        ON C1.pmid_citing = P.pmid;
        ''')
        with self.conn:
            self.cursor.execute(query)
            
        cocit_data = []
        for row in self.cursor:
            cocit_data.append(list(row))
        self.cocit_df = pd.DataFrame(cocit_data, columns=['citing', 'cited_1', 'cited_2', 'year'])
        logging.info(f'Found {len(self.cocit_df)} co-cited pairs of articles')
        
        self.cocit_grouped_df = self.cocit_df.groupby(['cited_1', 'cited_2', 'year']).count().reset_index()
        self.cocit_grouped_df = self.cocit_grouped_df.pivot_table(index=['cited_1', 'cited_2'], 
                                                          columns=['year'], values=['citing']).reset_index()
        self.cocit_grouped_df = self.cocit_grouped_df.replace(np.nan, 0)
        self.cocit_grouped_df['total'] = self.cocit_grouped_df.iloc[:, 2:].sum(axis=1)
        self.cocit_grouped_df = self.cocit_grouped_df.sort_values(by='total', ascending=False)
        
        self.CG = nx.Graph()
        for el in analyzer.cocit_grouped_df[['cited_1', 'cited_2', 'total']].values.astype(int):
            self.CG.add_edge(el[0], el[1], weight=el[2])

    def load_citation_stats(self, cit_stats_file):        
        logging.info('Started loading citation stats')
        self.cit_df = pd.read_csv(cit_stats_file, names=['pmid', 'year', 'count']).fillna(0)
        logging.info('Done loading citation stats')

        # XXX Using set here improves performance drastically!
        pmids_set = set(self.pmids)
        self.cit_df = self.cit_df.loc[np.logical_and([i in pmids_set for i in self.cit_df['pmid']], 
                                                     self.cit_df['year'] > 0)]
        logging.info('Done filtering')

        self.cit_df[['pmid']] = self.cit_df[['pmid']].applymap(np.uint32)
        self.cit_df[['year', 'count']] = self.cit_df[['year', 'count']].applymap(np.uint16)
        logging.info('Done transformation')

        self.cit_df = self.cit_df.pivot(index='pmid', columns='year', values='count').reset_index().replace(np.nan, 0)
        self.cit_df['total'] = self.cit_df.iloc[:, 1:].sum(axis = 1)
        self.cit_df = self.cit_df.sort_values(by='total', ascending=False)
        logging.info('Done aggregation')
        
        logging.info(f"Loaded citation stats for {len(self.cit_df)} of {len(self.pmids)} articles. " +
                    "Others may either have zero citations or be absent in the local database.")

            
    def plot_total_citations(self):
        plt.figure(figsize=(10, 5))
        ax = self.cit_df['total'].plot.bar()
        ax.set_xticklabels([])
        ax.set_xlabel('Articles')
        ax.set_ylabel('Number of citations')
    
    def search(self, *terms):
        print('TODO: handle queries which return more than 1000000 items')
        print('TODO: use local database instead of PubMed API')
        self.terms = [t.lower() for t in terms]
        query=' '.join(terms)
        handle = Entrez.esearch(db='pubmed', retmax='1000000', retmode='xml', term=query)
        self.pmids = [int(pmid) for pmid in Entrez.read(handle)['IdList']]
        logging.info(f'Found {len(self.pmids)} articles about {terms}')       
        
    def top_cited_papers(self, threshold=0.05):
        ids = self.cit_df.iloc[:round(len(self.cit_df) * threshold), 0].values
        counts = self.cit_df.iloc[:round(len(self.cit_df) * threshold), -1].values
        urls = [PUBMED_ARTICLE_BASE_URL + str(i) for i in ids]
        return zip(ids, urls, counts)

In [4]:
def build_data_source(df):
    # TODO: use d = ColumnDataSource(df)
    d = ColumnDataSource(data=dict(pmid=df['pmid'], title=df['title'], year=df['year'], total=df['total'],
                                   size=np.log(df['total']) / 10, pos=np.random.random(size=len(df))))
    return d

In [ ]:
TOOLS = "hover,pan,tap,wheel_zoom,box_zoom,reset,save"

def serve_scatter_article_layout(source, title, year_range=None):
    callback = CustomJS(args=dict(source=source, base=PUBMED_ARTICLE_BASE_URL), code="""
        var data = source.data, selected = source.selected.indices;
        if (selected.length == 1) {
            // only consider case where one glyph is selected by user
            selected_id = data['pmid'][selected[0]]
            for (var i = 0; i < data['pmid'].length; ++i){
                if(data['pmid'][i] == selected_id){
                    window.open(base + data['pmid'][i], '_blank');
                }
            }
        }
    """)

    p = figure(tools=TOOLS, toolbar_location="above", plot_width=960, plot_height=400, x_range=year_range, title=title)
    p.xaxis.axis_label = 'Year'
    p.hover.tooltips = [
        ("PMID", '@pmid'),
        ("Title", '@title'),
        ("Year", '@year'),
        ("Cited by", '@total paper(s) total')
    ]
    p.js_on_event('tap', callback)

    p.circle(x='year', y='pos', fill_alpha=0.2, source=source, radius='size')
    
    return p

def serve_citation_dynamics_layout():
    def update(b):
        try:
            pmid = int(text.value)
            data = analyzer.cit_df[analyzer.cit_df['pmid'] == pmid]
            if len(data) == 1:
                x = data.columns[1:-1].values.astype(int)
                y = data[x].values[0]
                bar.data_source.data = {'x': x, 'y': y}
            else:
                text.value = 'Bad ID'
            push_notebook(handle=h)
        except ValueError:
            text.value = ''

    title = "Number of Citations per Year"

    p = figure(tools=TOOLS, toolbar_location="above", plot_width=960, plot_height = 300, title=title)
    p.xaxis.axis_label = "Year"
    p.yaxis.axis_label = "Number of citations"
    p.hover.tooltips = [
        ("Year", "@x"),
        ("Cited by", "@y paper(s) in @x"),
    ]

    d = ColumnDataSource(data=dict(x=[], y=[]))
    bar = p.vbar(x='x', width=0.8, top='y', source=d, color='#A6CEE3', line_width=3)
    
    text = widgets.Text(
        value='',
        placeholder='Enter PMID',
        description='PMID:',
        disabled=False
    )

    button = widgets.Button(
        description='Show',
        disabled=False,
        button_style='info',
        tooltip='Show'
    )
    button.on_click(update)

    panel = widgets.HBox([text, button])

    display(panel)
    h = show(p, notebook_handle=True)
    
    return p, h, panel

In [ ]:
from collections import Counter

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

nltk.download('stopwords')

NUM_MOST_COMMON = 5

def get_ngrams(string):
    stop_words = set(stopwords.words('english')) 
    tokens = list(filter(lambda s: any(c.isalpha() for c in s), string.lower().replace(',', '').replace('.', '').split(' ')))
    tokens = [t for t in tokens if t not in stop_words and t not in analyzer.terms]
    ngrams = list(tokens)
    for t1, t2 in zip(tokens[:-1], tokens[1:]):
        ngrams.append(t1 + ' ' + t2)
    for t1, t2, t3 in zip(tokens[:-2], tokens[1:-1], tokens[2:]):
        ngrams.append(t1 + ' ' + t2 + ' ' + t3)
    return ngrams

def get_most_common_ngrams(titles):
    ngrams = []
    for title in titles:
        ngrams.extend(get_ngrams(title))
    most_common = {}
    for ngram, cnt in Counter(ngrams).most_common(NUM_MOST_COMMON):
        most_common[ngram] = cnt / len(titles)
    return most_common

[nltk_data] Downloading package stopwords to /Users/oleg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
analyzer = KeyPaperAnalyzer()
analyzer.search(*SEARCH_TERMS)
analyzer.load_publications()
analyzer.pub_df.head()

TODO: handle queries which return more than 1000000 items
TODO: use local database instead of PubMed API


2019-05-02 18:47:02,987 INFO: Found 271764 articles about ('human', 'aging')
2019-05-02 18:47:02,988 INFO: Loading publication data
2019-05-02 18:48:50,266 INFO: Found 271000 publications in the local database


,pmid,title,year
0,7,Maturation of the adrenal medulla--IV. Effects...,1975.0
1,54,Phospholipase A2 as a probe of phospholipid di...,1975.0
2,538,Effect on aging on plasma renin and aldosteron...,1975.0
3,561,Regional study of acid hydrolases and lysosoma...,1975.0
4,717,Gerovital-H3--a review of the literature.,1975.0


In [ ]:
# In case this command is too long, you can stop and start DB
# pg_ctl -D /usr/local/var/postgres stop -s -m fast
# pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
analyzer.load_cocitations()
analyzer.cocit_df.head()

2019-05-02 18:48:50,422 INFO: Calculating co-citations for selected articles
2019-05-02 19:00:54,591 INFO: Found 15204399 co-cited pairs of articles


In [ ]:
# %%time

# # Optimization experiments according to https://pgday.ru/files/pgmaster14/max.boguk.query.optimization.pdf
# # SELECT * FROM test WHERE id<10000 AND val IN (1,...,1000)
# # SELECT * FROM test JOIN (VALUES (1),...,(10)) AS v(val) USING (val) where id<10000;

# analyzer.conn.rollback()
# logging.info(f'GC {gc.collect()} object(s)')

# logging.info('original')
# # Random permutation with different values
# perm_ids = [analyzer.pmids[i] for i in np.random.permutation(len(analyzer.pmids))][:int(len(analyzer.pmids)/2)]
# query = '''
# SELECT C1.pmid_citing, C1.pmid_cited, C2.pmid_cited, P.year
# FROM Citations C1
# JOIN Citations C2
# ON C1.pmid_citing = C2.pmid_citing AND C1.pmid_cited < C2.pmid_cited
# JOIN Publications P
# ON C1.pmid_citing = P.pmid
# WHERE C1.pmid_cited = ANY(%s) AND C2.pmid_cited = ANY(%s)
# LIMIT 10000;
# '''
# analyzer.cursor.execute(query, (perm_ids, perm_ids,))
# cocit_data = []
# for row in analyzer.cursor:
#     cocit_data.append(list(row))
# print(len(cocit_data))
# analyzer.conn.rollback()

# logging.info('done')


# analyzer.conn.rollback()
# logging.info(f'GC {gc.collect()} object(s)')

# logging.info('query1')
# # Random permutation with different values
# perm_ids = [analyzer.pmids[i] for i in np.random.permutation(len(analyzer.pmids))][:int(len(analyzer.pmids)/2)]
# values = ', '.join(['({})'.format(i) for i in sorted(perm_ids)])
# query1 = re.sub('\$VALUES\$', values, '''
# SELECT C1.pmid_citing, C1.pmid_cited, C2.pmid_cited, P.year
# FROM Citations C1
# JOIN (VALUES $VALUES$) AS C1T(pmid_cited) ON (C1.pmid_cited = C1T.pmid_cited)
# JOIN Citations C2
# JOIN (VALUES $VALUES$) AS C2T(pmid_cited) ON (C2.pmid_cited = C2T.pmid_cited)
# ON C1.pmid_citing = C2.pmid_citing AND C1.pmid_cited < C2.pmid_cited
# JOIN Publications P
# ON C1.pmid_citing = P.pmid
# LIMIT 10000;
# ''')

# analyzer.cursor.execute(query1)
# cocit_data = []
# for row in analyzer.cursor:
#     cocit_data.append(list(row))
# print(len(cocit_data))

# logging.info('done')


In [ ]:
analyzer.load_citation_stats(cit_stats_file='../citations_per_year.csv')
analyzer.cit_df.head()

In [ ]:
df_all = pd.merge(analyzer.pub_df, analyzer.cit_df, on='pmid')
df_all.head()

## Subtopics a.k.a. Clusters in the Co-citation Graph

In [ ]:
import community
logging.info(f'Louvain community clustering of co-citation graph')
p = community.best_partition(analyzer.CG)
components = set(p.values())
logging.info(f'Found {len(components)} components')
# q = list(greedy_modularity_communities(analyzer.CG))

In [ ]:
GRANULARITY = 0.1
logging.info(f'Merging components smaller than {GRANULARITY} to "Other" component')
threshold = int(GRANULARITY * len(components))
comp_to_merge = {com: sum([p[node] == com for node in p.keys()]) <= threshold for com in components}
other_comp = len(components) - sum(comp_to_merge.values())
pm = {k: v if not comp_to_merge[v] else other_comp for k, v in p.items()}
logging.info(f'Processed {len(set(pm.values()))} components')

In [ ]:
import networkx as nx

logging.info('Visualize components')
plt.figure(figsize=(10, 10)) 
size = float(len(components))
pos = nx.spring_layout(analyzer.CG)
cmap = plt.cm.get_cmap('nipy_spectral', len(set(pm.values())))
for i, com in enumerate(components):
    list_nodes = [node for node in pm.keys() if pm[node] == com]
    nx.draw_networkx_nodes(analyzer.CG, pos, list_nodes, node_size = 20, node_color = [cmap(i)])

nx.draw_networkx_edges(analyzer.CG, pos, alpha=0.5)
plt.show()

In [ ]:
df_comp = pd.Series(pm).reset_index().rename(columns={'index': 'pmid', 0: 'comp'})
df_all = pd.merge(df_all, df_comp, on='pmid')
df_all.head()

In [ ]:
years = df_all.columns.values[3:-2].astype(int)
min_year, max_year = np.min(years), np.max(years)

In [ ]:
n_comps = df_all['comp'].nunique()
ds = [None] * n_comps
layouts = [None] * n_comps
most_common = [None] * n_comps
for c in range(n_comps):
    ds[c] = build_data_source(df_all[df_all['comp'] == c])
    most_common[c] = dict(get_most_common_ngrams(df_all[df_all['comp'] == c]['title'].values))
    kwd = ', '.join([f'{k} ({v:.2f})' for k, v in most_common[c].items()])
    title = f'Subtopic #{c + 1}: {kwd}'
    layouts[c] = serve_scatter_article_layout(ds[c], title, year_range=[min_year, max_year])
    show(layouts[c])

## Top Cited Papers Overall

In [ ]:
df_all = df_all.sort_values(by='total', ascending=False)

In [ ]:
THRESHOLD = 0.1 # 10 %

In [ ]:
ds_top = build_data_source(df_all.iloc[:round(len(analyzer.cit_df) * THRESHOLD), :])
layout_top = serve_scatter_article_layout(ds_top, 'Top cited papers', year_range=[min_year, max_year])
show(layout_top)

## Top Cited Papers for Each Year

In [ ]:
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap

max_gain_data = []
cols = df_all.columns[3:-2]
for i in range(len(cols)):
    max_gain = df_all[cols[i]].astype(int).max()
    if max_gain > 0:
        sel = df_all[df_all[cols[i]] == max_gain]
        max_gain_data.append([cols[i], sel['pmid'].values[0], 
                              sel['title'].values[0], max_gain])
        
max_gain_df = pd.DataFrame(max_gain_data, columns=['year', 'pmid', 'title', 'count'])
max_gain_df.head(20)

ds_max = ColumnDataSource(data=dict(year=max_gain_df['year'], pmid=max_gain_df['pmid'].astype(str),
                                   title=max_gain_df['title'], count=max_gain_df['count']))
colors = factor_cmap('pmid', palette=Category10[10], factors=max_gain_df['pmid'].astype(str).unique())

year_range = [1985, 2019]
p = figure(tools=TOOLS, toolbar_location="above", plot_width=960, plot_height=300, x_range=year_range, title='Max gain')
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Number of citations'
p.hover.tooltips = [
    ("PMID", '@pmid'),
    ("Title", '@title'),
    ("Year", '@year'),
    ("Cited by", '@count papers in @year')
]

p.vbar(x='year', width=0.8, top='count', fill_alpha=0.2, source=ds_max, fill_color=colors, line_color=colors)

show(p)

## Citation per Year Dynamics

In [ ]:
p, h, panel = serve_citation_dynamics_layout()